In [1]:
import cloudscraper
import pymysql
from openpyxl import Workbook
import requests as rq
import brotli
from bs4 import BeautifulSoup
import json
import re
import configparser
from urllib.parse import urlencode
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6", 
    "Connection":"keep-alive",
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "same-origin", 
    "Upgrade-Insecure-Requests": "?1", 
    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'  # 偽裝使用者代理
}

In [1]:
# file handle
def save_setting_ini(file_name, ini):
    config = configparser.ConfigParser()
    config['crawler'] = ini
    with open(file_name, 'w') as f:
        config.write(f)
        
def load_setting_ini(file_name):
    config = configparser.ConfigParser()
    config.read(file_name)
    data = {}
    data['former_part'] = config['crawler']['former_part']
    data['initial_pages'] = config['crawler']['initial_pages']
    data['latter_part'] = config['crawler']['latter_part']
    data['end_pages'] = config['crawler']['end_pages']
    data['next_url'] = config['crawler']['next_url']
    data['next_tag'] = config['crawler']['next_tag']
    data['domain'] = config['crawler']['domain']
    data['name'] = config['crawler']['name'].replace('%%','%')
    data['tag_section'] = config['crawler']['tag_section']
    data['tag_detail'] = config['crawler']['tag_detail']
    data['upper_limit'] = config['crawler']['upper_limit']
    data['data_download'] = config['crawler']['data_download']
    data_before ={}
    data_before['before_url'] = config['crawler']['before_url']
    data_before['before_data'] = config['crawler']['before_data']
    data_before['before_domain'] = config['crawler']['before_domain']
    data_before['before_method'] = config['crawler']['before_method']
    data_before['before_content_type'] = config['crawler']['before_content_type']
    db_settings={}
    db_settings_others={}
    db_settings['host'] = config['crawler']['host']
    if db_settings['host'] != '':
        db_settings['port'] = config['crawler'].getint('port')
        db_settings['user'] = config['crawler']['user']
        db_settings['password'] = config['crawler']['password']
        db_settings['db'] = config['crawler']['db']
        db_settings['charset'] = config['crawler']['charset']
        db_settings['autocommit'] = config['crawler']['autocommit']
        db_settings_others['table_name'] = config['crawler']['table_name']    
        db_settings_others['is_datetime'] = config['crawler']['is_datetime']
        db_settings_others['is_cover'] = config['crawler'].getboolean('is_cover')
    return data,data_before,db_settings,db_settings_others

SyntaxError: invalid syntax (2666586191.py, line 37)

In [ ]:
# 用網址來做換頁的爬蟲
class site_url_Spider:
    def __init__(self):
        self.s = rq.session()
        self.headers = headers
        self.datas=[]
        self.json=[]
        self.use_cloudscraper = True
        self.scraper = cloudscraper.CloudScraper(browser='chrome')
    def update_attr(self, name, headers_attr, url_attr, next_tag, tag_section, tag_detail, data, method):
        self.name = name
        # url_attr 用來組合要 request 的網址，依順序放入：(網址前半部分, 初始頁數, 網址後半部分, 結束頁數)
        self.former_part = url_attr['former_part']
        if url_attr['initial_pages'] == '':
            self.url = self.former_part
        else:
            self.initial_pages = int(url_attr['initial_pages'])
            self.latter_part = url_attr['latter_part']
            self.end_pages = int(url_attr['end_pages'])
            self.url = self.former_part+str(self.initial_pages)+self.latter_part
        # headers_attr 用來傳入 headers的相關參數，依順序放入：(headers['Host'], headers['Content-Type'])
        for i in headers_attr:
            if headers_attr[i] != None:
                headers[i]=headers_attr[i]
        self.tag_section = tag_section
        self.tag_detail = tag_detail
        self.next_tag = next_tag
        if data != None:
            if 'urlencoded' in headers_attr['Content-Type']:
                self.data=urlencode(data)
            elif 'json' in headers_attr['Content-Type']:
                self.data=json.dumps(data)
        if method != None:     
            self.method = method
        if next_tag != '':
            self.tag_upper_limit = int(next_tag[2])
    def before_craw(self):
        if self.use_cloudscraper:
            if self.method =='GET':
                r = self.scraper.get(self.url,headers = self.headers)
            if self.method =='POST':
                r = self.scraper.post(self.url,headers = self.headers, data = self.data)
            self.cookies = str(list(self.scraper.cookies))
        else:
            if self.method =='GET':
                r = self.s.get(self.url, headers = self.headers, data = self.data)
            if self.method =='POST':
                r = self.s.post(self.url, headers = self.headers, data = self.data)
            self.cookies = str(list(self.s.cookies))
        self.status_code = r.status_code
        self.html = r.text
        soup = BeautifulSoup(self.html, "html.parser")
        
    def test_parse(self, data_export, next_url):
        if data_export:
            self.datas=[]
            self.json=[]
        try:
            if self.use_cloudscraper:
                r = self.scraper.get(self.url,headers = self.headers)
            else:
                r = self.s.get(self.url,headers = self.headers)
        except rq.exceptions.RequestException:
            raise rq.exceptions.RequestException
        self.status_code = r.status_code
        self.html = r.text
        soup = BeautifulSoup(self.html, "html.parser")
        if next_url:
            try:
                url_section_list = soup.select(self.next_tag[0])
                url = url_section_list[self.next_tag[1]]['href']
                if re.search(r'http',url):
                    self.next_url = url
                else:
                    self.next_url = re.search(r'.*//.*?/',self.former_part).group(0)[:-1] + url
            except Exception as ex:
                print(ex)
                self.next_url = None
                    
        # tag_detail 預期要放這個：(tag_name, css_selector, 屬性(text href....))
        blocks = soup.select(self.tag_section)
        for (idx_block,block) in enumerate(blocks):
            data = []
            dic = {}
            for (idx_j,j) in enumerate(self.tag_detail):
                this = ''
                try:
                    obj = block.select(j[1])[j[2]]
                    if j[3] == 'text':
                        this = obj.text.strip()
                    else:
                        this = obj[j[3]].strip()
                except Exception as ex:
                    print(f"第 {idx_block+1} 個區塊，第 {idx_j+1} 個欲爬取的標籤 ( {j[1]} )無法獲取，跳過")
                    print(ex)
                data.append(this)
                dic[j[0]]=this
            self.datas.append(data)
            self.json.append(dic)
        if data_export:
            self.json = json.dumps(self.json,ensure_ascii=False)
    def url_loop_parse(self):
        self.datas=[]
        self.json=[]
        self.str = []
        self.html = "<div>當前未抓取任何頁面<div>"
        if self.initial_pages >= self.end_pages:
            start = self.end_pages
            end = self.initial_pages
        else:
            start = self.initial_pages
            end = self.end_pages
        for i in range(start, end+1):    
            self.url = self.former_part+str(i)+self.latter_part
            try:
                self.test_parse(data_export = False, next_url = False)
                self.str.append(f"第 {i} 頁爬取成功")
            except rq.exceptions.RequestException as e:
                self.str.append(f"第 {i} 頁爬取失敗，停止爬取")
                break
        self.json = json.dumps(self.json,ensure_ascii=False)
    def next_loop_parse(self):
        count=0
        self.datas=[]
        self.json=[]
        self.str = []
        self.url = self.former_part
        self.test_parse(data_export = False, next_url = True)
        while self.next_url != None and count<=self.tag_upper_limit:
            count=count+1
            self.url = self.next_url
            try:
                self.test_parse(data_export = False, next_url = True)
                self.str.append(f"第 {count} 頁爬取成功")
            except rq.exceptions.RequestException as e:
                self.str.append(f"第 {count} 頁爬取失敗，停止爬取")
                break
        self.json = json.dumps(self.json,ensure_ascii=False)
    def export_excel(self, path):
        wb = Workbook()
        sheet = wb.create_sheet(self.name, 0)
        sheet.append([i[0] for i in self.tag_detail])
        for i in self.datas:
            sheet.append(i)
        wb.save(path)

In [ ]:
# #gov_pur_crawler
# header_part={'Host':'web.pcc.gov.tw','Content-Type':None}
# url_attr={'former_part':'https://web.pcc.gov.tw/prkms/tender/common/basic/readTenderBasic?tenderEndDate=2022%2F06%2F01&orgName=&tenderName=&searchType=basic&d-49738-p=','initial_pages':'1','latter_part':'&firstSearch=false&pageSize=50&radProctrgCate=&tenderId=&orgId=&tenderStartDate=2022%2F06%2F01&tenderType=TENDER_DECLARATION&dateType=isDate&tenderWay=TENDER_WAY_ALL_DECLARATION&level_1=on','end_pages':'2'}
# ax= site_url_Spider()
# ax.update_attr("ptt_crawler",header_part,url_attr,'','div#printArea table.tb_01 tr',[('機關名稱','td',1,'text'),('標案案號','td',2,'text'),('連結','td a ',0,'href'),('傳輸次數','td',3,'text'),('招標方式','td',4,'text')],None,'')
# ax.url_loop_parse()        

In [ ]:
# #ptt_crawler
# header_part={'Host':'www.ptt.cc','Content-Type':None}
# url_attr={'former_part':'https://www.ptt.cc/bbs/LoL/index','initial_pages':'14386','latter_part':'.html','end_pages':'14380'}
# ax= site_url_Spider()
# ax.update_attr("ptt_crawler",header_part,url_attr, ("div.btn-group.btn-group-paging a",0,10),'div.r-ent',"[('標題','div.title',0,'text'),('作者','div.author',0,'text'),('日期','div.date',0,'text'),('連結','div.title a',0,'href')]",None, None)
# ax.next_loop_parse()

In [ ]:
# # inside_crawler
# header_part={'Host':'www.inside.com.tw','Content-Type':None}
# url_attr={'former_part':'https://www.inside.com.tw/tag/%E5%8F%B0%E7%A9%8D%E9%9B%BB?page=','initial_pages':'1','latter_part':'','end_pages':'4'}
# ax= site_url_Spider("inside_crawler",header_part,url_attr)
# ax.url_loop_parse('div.post_list.post_list-list_style div.post_list_item div.post_list_item_content',"[('標題','h3.post_title a','text'),('細項','p.post_description.js-auto_break_text','text'),('日期','ul.post_meta li.post_date','text'),('連結','h3.post_title a','href')]")

In [ ]:
# url = "https://www.inside.com.tw/tag/Apple"
# r= rq.get(url)
# next_tag = ("div.container ul.pagination li.pagination_item-next-wrapper a a", 0)
# soup = BeautifulSoup(r.text, "html.parser")
# url = soup.select(next_tag[0])